### Requirements
-----

In [1]:
import pandas as pd
pd.__version__

'1.5.3'

In [2]:
import numpy as np
np.__version__

'1.26.4'

In [3]:
import matplotlib
matplotlib.__version__

'3.9.0'

-----

### Make GIFs Great Again!

*Original blog post by [Sivakar Sivarajah](https://towardsdatascience.com/creating-beautiful-gif-with-python-for-your-data-analysis-ac50c618b559)*

In [4]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
plt.style.use('fivethirtyeight')

# Matplotlib XKCD Style Plots
# plt.xkcd()

In [5]:
df = pd.read_csv('data/data.csv', header=0, sep='|')
df.head()

,hits,visits,day,identifier,orders,amount,product_pages,direct_visit,organic_visit,paid_search_visit,email_visit
0,1084135,145634,2020-04-27,96,45986,3061233.89,707126,400028,260021,846,6
1,734485,111792,2020-04-30,96,53344,3271520.39,479824,255051,159261,431,0
2,2084615,182338,2020-04-08,96,11576,908171.75,1319358,675851,337172,37056,12
3,1133765,157161,2020-04-25,96,49829,3398320.87,720391,416621,237090,801,7
4,2473217,254864,2020-04-14,96,24317,2029124.65,1503301,736847,523907,75793,0


In [6]:
# filter on a single time series
df = df[(df.identifier==36)]

In [7]:
# transform day as a datetime object
df['day']=pd.to_datetime(df['day'])
# set day as an index 
df.set_index('day',drop=True, inplace=True)

In [8]:
df.head()

,hits,visits,identifier,orders,amount,product_pages,direct_visit,organic_visit,paid_search_visit,email_visit
day,,,,,,,,,,
2019-08-22,1633273,323468,36,28822,5.442457e+06,535614,246413,108930,143351,5095
2019-05-09,1722637,350463,36,30086,5.278870e+06,555791,264586,108284,125806,13444
2019-12-22,3732463,734510,36,71886,1.448650e+07,1235813,480195,245872,415754,20915
2020-04-25,3246407,510123,36,13570,2.951195e+06,949286,478942,263048,146732,9686
2020-02-19,1661310,345783,36,28985,5.066071e+06,557016,233672,105473,163391,8884


In [9]:
# resample dataframe on a week basis
df=df.resample('W').sum()
df.shape

(57, 10)

In [10]:
#Defining the start and end dates
START=df.index[0]
END=df.index[-1]

In [11]:
import gif

In [12]:
@gif.frame
def plot(df,date):
    
    maxi1=round(df.orders.max()*1.5)

    df=df.loc[df.index[0]:pd.Timestamp(date)]
    fig, (ax1) = plt.subplots(1,figsize=(5,3),dpi=100)
    ax1.plot(df.orders,marker='o', linestyle='--', linewidth=2,markersize=3, color='tab:green')

    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi1])
    ax1.tick_params(axis='x', labelsize=7)
    ax1.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    plt.title('Nb. of orders per week', fontsize=11)
    plt.tight_layout()


In [13]:
#### ANIMATION CREATION ####
frames = []
for date in pd.date_range(start = START, end = END,freq = '1W'):
    frame = plot(df,date)
    frames.append(frame)

In [14]:
#saving the GIF
gif.save(frames, "data/output1.gif", duration=150) # ms

In [15]:
#visualizing the GIF
from IPython.display import HTML
HTML('<img src="data/output1.gif" />')

In [16]:
@gif.frame
def plot_split(df,date,split_date='2019-12-31'):
    
    maxi1=round(df.orders.max()*1.5)

    df=df.loc[df.index[0]:pd.Timestamp(date)]
    fig, (ax1) = plt.subplots(1,figsize=(7,5),dpi=100)
    
    #2019
    if date <= pd.Timestamp(split_date):
        ax1.axvspan(START,date, alpha = 0.5, color = '#33FF92')
        ax1.text(pd.Timestamp('2019-05'),y= 10000, s = '2019',fontsize=11)

    #2020
    if (date > pd.Timestamp(split_date)):
        ax1.axvspan(pd.Timestamp(split_date),date, alpha = 0.5,   color = '#F933FF')
        ax1.text(pd.Timestamp('2020-01'),y= 10000, s = '2020',fontsize=11)
    
    ax1.plot(df.orders,marker='o', linestyle='--', linewidth=2,markersize=3, color = 'tab:orange')
    ax1.set_ylabel('orders',color = 'blue',fontsize=7)    

    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi1])
    ax1.tick_params(axis='x', labelsize=7)
    ax1.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    plt.title('2019/2020 Split',fontsize=11)
    plt.tight_layout()

frames = []
for date in pd.date_range(start = START, end = END,freq = '1W'):
    frame = plot_split(df,date)
    frames.append(frame)

gif.save(frames, "data/output2.gif", duration=150)

HTML('<img src="data/output2.gif" />')

In [17]:
@gif.frame
def plot(df,date):
    
    maxi1=round(df.orders.max()*1.5)
    maxi2=round(df.amount.max()*1.5)
        
    df=df.loc[df.index[0]:pd.Timestamp(date)]
    fig, ax1 = plt.subplots(1,figsize=(7,5),dpi=100)
    
    ax1.plot(df.orders, color = 'tab:orange',marker='o', linestyle='--', linewidth=2,markersize=3)
    ax1.set_ylabel('orders',color = 'tab:orange',fontsize=11)

    ax2 = ax1.twinx()
    ax2.plot(df.amount, color = 'tab:blue',marker='o', linestyle='--', linewidth=2,markersize=3)
    ax2.set_ylabel('amount',color = 'tab:blue',fontsize=11)
    
    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi1])
    ax1.tick_params(axis='x', labelsize=7)
    ax1.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    ax2.set_ylim([0, maxi2])
    ax2.tick_params(axis='x', labelsize=7)
    ax2.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    plt.title('Nb. of orders & Amount in €', fontsize=11)
    plt.tight_layout()

frames = []
for date in pd.date_range(start = START, end = END,freq = '1W'):
    frame = plot(df,date)
    frames.append(frame)

gif.save(frames, "data/output3.gif", duration=150)

HTML('<img src="data/output3.gif" />')

In [18]:
@gif.frame
def plot(df,date):
    
    maxi1=round(df.orders.max()*1.2)
    maxi2=round(df.amount.max()*1.2)        
    maxi3=round(df.product_pages.max()*1.2)        

    df=df.loc[df.index[0]:pd.Timestamp(date)]
    fig, (ax1,ax2,ax3) = plt.subplots(3,figsize=(11,7),dpi=100)
    
    ax1.plot(df.orders,marker='o', linestyle='--', linewidth=2,markersize=3, color='g')
    ax1.set_ylabel('orders',color = 'green',fontsize=11)
    
    ax2.plot(df.amount,marker='o', linestyle='--', linewidth=2,markersize=5, color='r')
    ax2.set_ylabel('amount',color = 'red',fontsize=11)
    
    ax3.plot(df.product_pages,marker='o', linestyle='--', linewidth=2,markersize=3, color='b')
    ax3.set_ylabel('product pages',color = 'blue',fontsize=11)

    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi1])
    ax1.tick_params(axis='x', labelsize=7)
    ax1.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    ax2.set_xlim([START, END])
    ax2.set_ylim([0, maxi2])
    ax2.tick_params(axis='x', labelsize=7)
    ax2.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    ax3.set_xlim([START, END])
    ax3.set_ylim([0,maxi3])
    ax3.tick_params(axis='x', labelsize=7)
    ax3.tick_params(axis='y', labelsize=7 , rotation=33)    
    ax3.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))

    plt.tight_layout()

frames = []
for date in pd.date_range(start = START, end = END,freq = '1W'):
    frame = plot(df,date)
    frames.append(frame)

gif.save(frames, "data/output4.gif", duration=150)
    
HTML('<img src="data/output4.gif" />')